## <font color=green>0. Imports</font>

In [1]:
import pandas as pd
from ast import literal_eval

In [2]:
## Importing the object class
df_object_class = pd.read_csv("object_class.csv")
## Importing the category class and back4app csv
df_painting_back4app = pd.read_csv("painting_back4app.csv")
## Importing the painting class and back4app csv
df_category_back4app = pd.read_csv("category_back4app.csv")

### <font color=##5A5A5A>5.2 Mapping Painting Pointer onto the Object Class</font>

In [3]:
## Ensuring that the coords array is in fact an array
df_object_class['coords'] = df_object_class['coords'].apply(literal_eval)

In [4]:
df_painting_back4app.head(2)

,objectId,updatedAt,createdAt,ACL,image_width,techniques,suggested_bg_color,artist,image_thumbnail,image_height,colors,publication_year,painting_id,title,object_labels
0,nl7LbVa9Zt,Fri Apr 28 2023 13:11:21 GMT+0200 (Central Eur...,Fri Apr 28 2023 13:11:21 GMT+0200 (Central Eur...,NaN,4833,Olie på træ,#888888,Jacob Weyer,https://iip-thumb.smk.dk/iiif/jp2/1r66j221x_KM...,3407,"['#dddddd', '#444444', '#555555', '#333333', '...",NaN,KMS638,Slagbillede,"[""window"",""lake"",""flag"",""horse"",""donkey"",""sky""..."
1,ZzHAPkDjqS,Fri Apr 28 2023 13:11:21 GMT+0200 (Central Eur...,Fri Apr 28 2023 13:11:21 GMT+0200 (Central Eur...,NaN,3131,Olie på lærred,#908d41,Nicolai Abildgaard,https://iip-thumb.smk.dk/iiif/jp2/kk91fp68v_KM...,3738,"['#222222', '#3a351f', '#908d41', '#382510', '...",1999,KMS595,"En potuaner, hvis reformforslag er billiget, b...","[""window"",""spear"",""spear"",""sword"",""demon""]"


In [5]:
painting_id_dict = dict(zip(df_painting_back4app['painting_id'], df_painting_back4app['objectId']))

In [6]:
df_objects_back4app = df_object_class.copy()

In [7]:
# map object labels to categories
df_objects_back4app['painting_id_back4app'] = df_objects_back4app['painting_id'].map(painting_id_dict)

In [8]:
# create a function that modifies the last value of a dictionary based on the function argument x
def modify_dict_last_value(sample_dict, x):
    keys = list(sample_dict.keys())
    values = list(sample_dict.values())
    values[-1] = x
    modified_dict = dict(zip(keys, values))
    return modified_dict

In [9]:
pointer_object = {'__type': 'Pointer', 'className': 'Painting', 'objectId': 'abc123'}
df_objects_back4app['painting_pointer'] = df_objects_back4app['painting_id_back4app'].apply(lambda x: modify_dict_last_value(pointer_object, x))

### <font color=##5A5A5A>5.3 Mapping Category Pointers onto the Object Class</font>

In [10]:
category_id_dict = dict(zip(df_category_back4app['category_name'], df_category_back4app['objectId']))

In [11]:
# map object labels to categories
df_objects_back4app['category_id'] = df_objects_back4app['category_name'].map(category_id_dict)

In [12]:
category_point = {'__type': 'Pointer', 'className': 'Category', 'objectId': 'abc123'}
df_objects_back4app['category_pointer'] = df_objects_back4app['category_id'].apply(lambda x: modify_dict_last_value(category_point, x))

### <font color=##5A5A5A>5.4 Resetting indexes of dataframes</font>

In [13]:
df_objects_back4app = df_objects_back4app.reset_index(drop= True)

In [14]:
## checking painting pointer for random object
pd.options.display.max_colwidth = 100
df_objects_back4app.query('object_id == "d29403c7d5aa4e53a682d2086d361beb"')['painting_pointer']

Series([], Name: painting_pointer, dtype: object)

In [15]:
## checking category pointer for random object
pd.options.display.max_colwidth = 100
df_objects_back4app.query('object_id == "d29403c7d5aa4e53a682d2086d361beb"')['category_pointer']

Series([], Name: category_pointer, dtype: object)

In [16]:
df_objects_final = df_objects_back4app.copy()

In [17]:
df_objects_final = df_objects_final.filter(['label_text', 'coords', 'image_thumbnail','category_name', 'painting_id_back4app', 'painting_pointer', 'category_id', 'category_pointer'], axis=1)

In [18]:
df_objects_final.to_json('object_class.json', orient="table")

In [19]:
df_objects_final.shape

(10142, 8)